In [ ]:
import pathlib
import sys

import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import autocrit

In [ ]:
sys.path.append("..")

import shared.format
import shared.tools

In [ ]:
plt.rcParams["font.size"] = shared.format.FONTSIZE

In [ ]:
import autocrit

In [ ]:
import autocrit_tools

In [ ]:
pickle_dir = pathlib.Path("data") / "linear-results"

analytical_cp_df = pd.read_pickle(pickle_dir / "analytical_cp_df.pkl")

def compute_index(spectrum, eigval_cutoff=1e-5):
    return np.mean(spectrum < -eigval_cutoff)

analytical_cp_df["index"] = analytical_cp_df["hessian_spectrum"].apply(compute_index)

In [ ]:
import string

uppercase_letters = string.ascii_uppercase

In [ ]:
labelsize = "x-large"
titlesize = "xx-large"

fig, axs = plt.subplots(figsize=(18, 12), nrows=2, ncols=3)
[(ax.set_xlabel("Iteration", size=labelsize),
  ax.set_ylabel(r"$\|\|\nabla L(\theta)\|\|^2$", size=labelsize))
  for ax in axs[0]]

[(ax.set_xlabel("Index", size=labelsize),
  ax.set_ylabel(r"$L(\theta)$", size=labelsize))
  for ax in axs[1]]

axs[0, 0].set_title("Newton-MR", size=titlesize)
axs[0, 1].set_title("Damped Newton", size=titlesize)
axs[0, 2].set_title("BTLS-GNM", size=titlesize)

[ax.scatter(
    analytical_cp_df["index"], analytical_cp_df["cost"], color="gray",
    label="analytical")
 for ax in axs[1]]

axs[-1, -1].legend(loc="lower right");

[shared.tools.add_panel_label(letter, ax, pos=(-0.1, 1.1), size=titlesize)
 for letter, ax in zip(uppercase_letters, axs.flatten())]

plt.tight_layout();
plt.savefig("perf-comparison.pdf")